In [8]:
# helper library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os, re, cv2
import json, shutil
import collections
import base64,random
from tqdm import tqdm 
from typing import Dict, List
import matplotlib.pyplot as plt
from collections import defaultdict
from glob import glob
plt.style.use("seaborn")
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns',50)

# Image
from PIL import Image,ImageDraw
import cv2 as cv


# 1. Get sunny/day annotation

In [2]:
# annotation 파일명 lsit 생성 함수
def get_annotation(data_path,annotation_list=[]):
    '''
    data_path: data directory path
    annotation_list: list of annotaion(label)
    '''
    for file in os.listdir(data_path):
        path = os.path.join(data_path+'/',file)
        if os.path.isfile(path):
            if file.endswith('json'):
                with open(path,'r',encoding="UTF-8") as annotation:
                    annotation_list.append(json.load(annotation))
        elif os.path.isdir(path) :
            get_annotation(path,annotation_list)
    return annotation_list

In [1]:
# dataset의 개수를 파악하는 function
def number_of_data(data_path,number_of_image = 0 ,number_of_annotation = 0,total_data = 0):
    '''
    data_path: data directory path
    number_of_image: number of image
    number_of_annotation: number of anntation
    '''
    for file in os.listdir(data_path):
        path = os.path.join(data_path+'/',file)
        if os.path.isfile(path):
            total_data +=1
            if file.endswith('json'):
                number_of_annotation += 1
            elif file.endswith('png'):
                number_of_image += 1
        elif os.path.isdir(path) :
            number_of_image,number_of_annotation,total_data = number_of_data(path,number_of_image,number_of_annotation,total_data)
    return number_of_image,number_of_annotation,total_data

In [3]:
# 주간/맑음 파일 선별
def move_file(data_path,move_list=[],move_path = 'C:/Users/Lee/Desktop/AIFFEL/Aimmo_dataset'):
    '''
    data_path: data directory path
    annotation_list: list of annotaion(label)
    '''
    for file in os.listdir(data_path):
        path = os.path.join(data_path+'/',file)
        if os.path.isfile(path):
            if file.endswith('json'):
                with open(path,'r',encoding="UTF-8") as annotation:
                    anno = json.load(annotation)
                    if anno['weather'] == 'sunny' and anno['time'] == 'day':
                        move_list.append(path)
                        move_list.append(path.replace('_Bbox_GT.json','.png'))
                                         
        elif os.path.isdir(path) :
            move_file(path,move_list)
    for i in move_list:
        shutil.move(i,move_path) 
    
    return move_list

In [4]:
# 주간/맑음 annotation
def sunny_day_annotation(move_path,annotation_list=[]):
    for file in os.listdir(move_path):
        if file.endswith('json'):
            with open(move_path+'/'+file,'r',encoding="UTF-8") as annotation:
                annotation_list.append(json.load(annotation))
    return annotation_list

## 2. Annotation adjustment

In [88]:
# label/attribute drop 
def adjust_annotation(path):
    '''
    1.car,bus,truck,pedestrian,other 을 제외한 class 제거
    2. pixel area가 625 이하인 bbox 제거
    '''
    count=0
    for file in os.listdir(path):
        if file.endswith('json'):
            id  = []
            with open(path+'/'+file, 'r', encoding="UTF-8") as annotations:
                anno = json.load(annotations)
                for index,annotation in enumerate(anno['annotations']):
                    if (annotation['label'] == 'traffic_sign') or (annotation['label'] == 'traffic_light') or (annotation['label'] == 'ptw') or (annotation['label'] == 'rider') or (annotation['label'] == 'car_number') or (annotation['label'] == 'movable_object') or (annotation['label'] == 'unknown'):
                        id.append(index)
                    elif annotation['area']<625 :
                        id.append(index)

                id.reverse()
                for index in id:
                    del anno['annotations'][index]
            try:
                with open(path+'/'+file, 'w', encoding='utf-8') as ch_annotations:
                    json.dump(anno, ch_annotations, indent="\t")
            except:
                count+=1
    return count

In [89]:
def make_class(path):
    '''
    - label/attribute 통합 class 생성
    '''
    
    count = 0
    for file in os.listdir(path):
        if file.endswith('json'):
            id  = []
            with open(path+'/'+file, 'r', encoding="UTF-8") as annotations:
                anno = json.load(annotations)

                for index,annotation in enumerate(anno['annotations']):
                    if (annotation['label'] == 'face') or (annotation['label'] == 'pedestrian'):
                        annotation['class'] = 'pedestrian'
                    elif annotation['attribute'] == 'car':
                        annotation['class'] = 'car'
                    elif (annotation['attribute'] == 'bus_l') or(annotation['attribute'] == 'bus_s'):
                        annotation['class'] = 'bus'
                    elif (annotation['attribute'] == 'truck_l') or(annotation['attribute'] == 'truck_s'):
                        annotation['class'] = 'truck'
                    else:
                        annotation['class'] = 'other'
                   
            try:
                with open(path+'/'+file, 'w', encoding='utf-8') as ch_annotations:
                    json.dump(anno, ch_annotations, indent="\t")
            except:
                count+=1
    return count

In [55]:
print(human)
print(car)
print(bus)
print(truck)
print(other)

31405
129702
17989
30509
3018


In [92]:
def get_annotataion_df(annotations):
    
    anno_list = []
    name_list = []

    for index,annotation in enumerate(annotations):
        name = annotation['filename']
        if annotation['weather'] == 'sunny' and annotation['time'] == 'day':
            for data in annotation['annotations']:
                anno_list.append(data)
                name_list.append(name)

    anno_df = pd.DataFrame(anno_list)
    anno_df['ano_name'] = name_list
    return  anno_df

In [93]:
anno_df = get_annotataion_df(annotations)